In [2]:
#import pyarrow.feather as feather
import pandas as pd
import numpy as np
import os
import seaborn as sns
sns.set(color_codes=True)
from numpy.random import seed
#from scipy.io import netcdf
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

from math import sqrt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers

In [3]:
# it is better for visualisation if the columns date type (not int or str)
# when save df to csv, if the column names are date, they are saved as str
def convert_col_to_date(columns):
    
    # convert the columns type
    #columns = df.columns
    new_columns = []
    for col in columns:
        try:
            new_columns.append(pd.to_datetime(col).date())
        except: new_columns.append(col)
    return new_columns

In [3]:
# import freq df
freq_level =  pd.read_csv(f"dataset/modeling/freq_level.csv").set_index("proj_id")
print(freq_level.shape)
print(" ")
print("details: ")
print(f"daily_freq: {freq_level[freq_level['daily']].shape}")
print(f"mid_weekly_freq: {freq_level[freq_level['mid_weekly']].shape}")
print(f"weekly_freq: {freq_level[freq_level['weekly']].shape}")
print(f"fortnight_freq: {freq_level[freq_level['fortnight']].shape}")
print(f"low_freq_freq: {freq_level[freq_level['low_freq']].shape}")

freq_level.sort_values(["freq", "length"]).head()

(12326, 7)
 
details: 
daily_freq: (3028, 7)
mid_weekly_freq: (957, 7)
weekly_freq: (5713, 7)
fortnight_freq: (1658, 7)
low_freq_freq: (970, 7)


,daily,mid_weekly,weekly,fortnight,low_freq,length,freq
proj_id,,,,,,,
BY_6169,True,False,False,False,False,161,1
BY_6168,True,False,False,False,False,162,1
SH_10L58130001,True,False,False,False,False,162,1
NI_40004396,True,False,False,False,False,200,1
BB_39515055,True,False,False,False,False,224,1


In [6]:
# import t s and convert into date type
grouped_t_s = pd.read_csv(f"dataset/modeling/grouped_t_s.csv").set_index("proj_id")
grouped_t_s.columns = convert_col_to_date(grouped_t_s.columns)
print(grouped_t_s.shape)
grouped_t_s.head()

(12326, 6546)


,length,freq,1995-01-01,1995-01-02,1995-01-03,1995-01-04,1995-01-05,1995-01-06,1995-01-07,1995-01-08,...,2012-11-21,2012-11-22,2012-11-23,2012-11-24,2012-11-25,2012-11-26,2012-11-27,2012-11-28,2012-11-29,2012-11-30
proj_id,,,,,,,,,,,,,,,,,,,,,
BB_25470023,567,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,79.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BB_25470024,566,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,79.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BB_25480025,567,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,35.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BB_25480026,569,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,81.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BB_25481088,5019,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.27,37.26,37.27,37.26,37.29,37.29,37.3,37.31,37.33,37.32


In [8]:
grouped_t_s["length"][2000:3500].mean()

1071.9253333333334

In [19]:
# import ave_temp 
ave_temp = pd.read_csv(f"dataset/modeling/ave_temp.csv").set_index("proj_id")
ave_temp.columns = convert_col_to_date(ave_temp.columns)
print(ave_temp.shape)
ave_temp.head()


(12326, 6546)


,length,freq,1995-01-01,1995-01-02,1995-01-03,1995-01-04,1995-01-05,1995-01-06,1995-01-07,1995-01-08,...,2012-11-21,2012-11-22,2012-11-23,2012-11-24,2012-11-25,2012-11-26,2012-11-27,2012-11-28,2012-11-29,2012-11-30
proj_id,,,,,,,,,,,,,,,,,,,,,
BB_25470023,567,7,14.0,-8.0,-26.0,-63.0,-65.0,-80.0,-43.4286,-53.4286,...,24.4286,33.7143,39.2857,43.4286,47.2857,51.5714,54.0000,54.4286,51.5714,48.7143
BB_25470024,566,7,14.0,-8.0,-26.0,-63.0,-65.0,-80.0,-43.4286,-53.4286,...,24.4286,33.7143,39.2857,43.4286,47.2857,51.5714,54.0000,54.4286,51.5714,48.7143
BB_25480025,567,7,17.0,-6.0,-27.0,-67.0,-63.0,-80.0,-42.8571,-52.7143,...,22.5714,32.4286,37.8571,42.4286,46.1429,50.1429,52.7143,53.2857,51.0000,49.0000
BB_25480026,569,7,14.0,-8.0,-28.0,-65.0,-65.0,-81.0,-44.2857,-54.2857,...,24.5714,33.8571,39.4286,43.5714,47.4286,51.8571,54.2857,54.7143,51.7143,48.7143
BB_25481088,5019,1,17.0,-6.0,-26.0,-66.0,-62.0,-80.0,-74.0000,-52.0000,...,44.0000,59.0000,33.0000,49.0000,60.0000,62.0000,64.0000,48.0000,42.0000,19.0000


In [20]:
# import the cum_prec 
cum_prec=pd.read_csv(f"dataset/modeling/cum_prec.csv").set_index("proj_id")
cum_prec.columns = convert_col_to_date(cum_prec.columns)
print(cum_prec.shape)
cum_prec.head()

(12326, 6546)


,length,freq,1995-01-01,1995-01-02,1995-01-03,1995-01-04,1995-01-05,1995-01-06,1995-01-07,1995-01-08,...,2012-11-21,2012-11-22,2012-11-23,2012-11-24,2012-11-25,2012-11-26,2012-11-27,2012-11-28,2012-11-29,2012-11-30
proj_id,,,,,,,,,,,,,,,,,,,,,
BB_25470023,567,7,3.0,25.0,0.0,0.0,0.0,0.0,28.0,39.0,...,17.0,18.0,24.0,32.0,20.0,41.0,41.0,58.0,165.0,165.0
BB_25470024,566,7,3.0,25.0,0.0,0.0,0.0,0.0,28.0,39.0,...,17.0,18.0,24.0,32.0,20.0,41.0,41.0,58.0,165.0,165.0
BB_25480025,567,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,17.0,20.0,26.0,15.0,37.0,37.0,42.0,158.0,157.0
BB_25480026,569,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.0,17.0,22.0,29.0,18.0,40.0,40.0,62.0,177.0,178.0
BB_25481088,5019,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,3.0,6.0,4.0,22.0,0.0,12.0,122.0,3.0


In [21]:
###########################################################################################################

# functions

In [108]:
# create df for a t s (one row in grouped_t_s df)
def create_one_ts_df(row):
    bgr = row.name
    idx = row.dropna().index[2:]
    ts = row[idx]
    temp = ave_temp[idx].loc[bgr]
    prec = cum_prec[idx].loc[bgr]
    df = pd.DataFrame({"ts":ts,"temp": temp,"prec":prec})
    return df



# define the autoencoder network model
def autoencoder_model(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L1 = LSTM(16, activation='relu', return_sequences=True, 
              kernel_regularizer=regularizers.l2(0.00))(inputs)
    L2 = LSTM(4, activation='relu', return_sequences=False)(L1)
    L3 = RepeatVector(X.shape[1])(L2)
    L4 = LSTM(4, activation='relu', return_sequences=True)(L3)
    L5 = LSTM(16, activation='relu', return_sequences=True)(L4)
    output = TimeDistributed(Dense(X.shape[2]))(L5)    
    model = Model(inputs=inputs, outputs=output)
    return model

# find the outliers and save them in the df of a ts
def find_outlier(df):

    # split into train 80% and test sets, 
    train = df[:n_train]
    test = df[n_train:]

    # normalize the data
    X_train = scaler.fit_transform(train)
    X_test = scaler.transform(test)
    # reshape inputs for LSTM [samples, timesteps, features]
    X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    #print("Training data shape:", X_train.shape)
    X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
    #print("Test data shape:", X_test.shape)

    # create the autoencoder model
    model = autoencoder_model(X_train)
    model.compile(optimizer='adam', loss='mae')
    #model.summary()

    # fit the model to the data
    history = model.fit(X_train, X_train, epochs=nb_epochs, batch_size=batch_size,validation_split=0.05).history
    
    # compute the threshold
    X_pred = model.predict(X_train)
    X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
    Xtrain = X_train.reshape(X_train.shape[0], X_train.shape[2])
    scored = np.mean(np.abs(X_pred-Xtrain), axis = 1)
    thre = np.mean(scored) +4*np.std(scored)
    #print(thre)
    
    # find anomalies in original df
    pred_df  = model.predict(np.concatenate([X_train,X_test]))
    pred_df = pred_df.reshape(pred_df.shape[0], pred_df.shape[2])
    df["anomaly"] = np.mean(np.abs(pred_df -scaled_df), axis = 1)>thre


In [109]:
# global varieble
outlier = []
nb_epochs = 40
batch_size = 10
scaler = MinMaxScaler()
num_ts = len(grouped_t_s)

# #################################################################

In [110]:
a,b =9,12
T1 = datetime.now()
print(f"T1: {T1}")
for i in range(a,b):
    ts = grouped_t_s.iloc[2]
    #bgr = ts.name
    df = create_one_ts_df(ts)
    n_train = int(len(df)*0.7) 
    scaled_df = scaler.fit_transform(df.values)
    # fit the train dataset
    find_outlier(df)
    outlier.append(df["anomaly"])

print(f"{999}th ts done")
T2 = datetime.now()
print(f"T2: {T2}")
print(f"T2-T1: {T2-T1}")


T1: 2021-01-04 12:01:40.452509
Epoch 1/40
38/38 [==============================] - 5s 20ms/step - loss: 0.4108 - val_loss: 0.3380
Epoch 2/40
38/38 [==============================] - 0s 6ms/step - loss: 0.3760 - val_loss: 0.3055
Epoch 3/40
38/38 [==============================] - 0s 6ms/step - loss: 0.3308 - val_loss: 0.2654
Epoch 4/40
38/38 [==============================] - 0s 6ms/step - loss: 0.2722 - val_loss: 0.2096
Epoch 5/40
38/38 [==============================] - 0s 5ms/step - loss: 0.1587 - val_loss: 0.1605
Epoch 6/40
38/38 [==============================] - 0s 6ms/step - loss: 0.1396 - val_loss: 0.1462
Epoch 7/40
38/38 [==============================] - 0s 6ms/step - loss: 0.1353 - val_loss: 0.1412
Epoch 8/40
38/38 [==============================] - 0s 6ms/step - loss: 0.1327 - val_loss: 0.1385
Epoch 9/40
38/38 [==============================] - 0s 6ms/step - loss: 0.1351 - val_loss: 0.1412
Epoch 10/40
38/38 [==============================] - 0s 5ms/step - loss: 0.1384 - val_

38/38 [==============================] - 0s 6ms/step - loss: 0.3279 - val_loss: 0.2444
Epoch 4/40
38/38 [==============================] - 0s 5ms/step - loss: 0.2429 - val_loss: 0.1489
Epoch 5/40
38/38 [==============================] - 0s 5ms/step - loss: 0.1437 - val_loss: 0.1446
Epoch 6/40
38/38 [==============================] - 0s 5ms/step - loss: 0.1381 - val_loss: 0.1410
Epoch 7/40
38/38 [==============================] - 0s 5ms/step - loss: 0.1467 - val_loss: 0.1371
Epoch 8/40
38/38 [==============================] - 0s 6ms/step - loss: 0.1397 - val_loss: 0.1403
Epoch 9/40
38/38 [==============================] - 0s 8ms/step - loss: 0.1358 - val_loss: 0.1423
Epoch 10/40
38/38 [==============================] - 0s 10ms/step - loss: 0.1363 - val_loss: 0.1391
Epoch 11/40
38/38 [==============================] - 0s 11ms/step - loss: 0.1412 - val_loss: 0.1385
Epoch 12/40
38/38 [==============================] - 0s 6ms/step - loss: 0.1390 - val_loss: 0.1393
Epoch 13/40
38/38 [=======

In [113]:
# save the outlier df
outliers =pd.DataFrame(outlier) 
outliers.index=grouped_t_s.index[a:b]
outliers = outliers[sorted(outliers.columns)]


In [114]:
outliers

,2000-10-17,2000-11-22,2000-12-18,2001-01-11,2001-02-07,2001-03-07,2001-04-04,2001-05-01,2001-05-08,2001-05-15,...,2012-09-15,2012-09-22,2012-10-01,2012-10-08,2012-10-15,2012-10-22,2012-11-01,2012-11-08,2012-11-15,2012-11-22
proj_id,,,,,,,,,,,,,,,,,,,,,
BB_25501075,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
BB_25501076,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
BB_25510027,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [119]:
# detect the outliers in one ts
k=9
one_ts = pd.DataFrame((grouped_t_s.iloc[9].dropna(),outliers.iloc[0].dropna())).T
one_ts.columns= [one_ts.columns[0],"anomaly"]
one_ts[one_ts["anomaly"]==True]
#one_ts

,BB_25501075,anomaly
2011-08-01,43.66,True
2012-07-08,44.2,True


In [120]:
one_ts

,BB_25501075,anomaly
length,4368,NaN
freq,1,NaN
1996-11-26,43.73,NaN
1997-11-01,43.71,NaN
1997-11-02,43.71,NaN
...,...,...
2009-04-08,NaN,False
2009-04-22,NaN,False
2009-05-01,NaN,False
2009-05-08,NaN,False


# ###################################

In [ ]:
# apply the model on all ts, here 70% train 30%test, 40 epochs
a,b =2000,3500
T1 = datetime.now()
print(f"T1: {T1}")
for i in range(a,b):
    ts= grouped_t_s.iloc[i]
    df = create_one_ts_df(ts)
    n_train = int(len(df)*0.7) 
    scaled_df = scaler.fit_transform(df.values)
    # fit the train dataset
    find_outlier(df)
    outlier.append(df["anomaly"])

print(f"{999}th ts done")
T2 = datetime.now()
print(f"T2: {T2}")
print(f"T2-T1: {T2-T1}")

# save the outlier df
outliers3 =pd.DataFrame(outlier) 
outliers3.index=grouped_t_s.index[a:b]
outliers3 = outliers3[sorted(outliers3.columns)]
outliers3.to_csv("dataset/outliers/outliers3.csv")


T1: 2021-01-03 23:58:25.110562
Epoch 1/40
61/61 [==============================] - 7s 26ms/step - loss: 0.3382 - val_loss: 0.3322
Epoch 2/40
61/61 [==============================] - 0s 5ms/step - loss: 0.2718 - val_loss: 0.2153
Epoch 3/40
61/61 [==============================] - 0s 5ms/step - loss: 0.1676 - val_loss: 0.1211
Epoch 4/40
61/61 [==============================] - 0s 5ms/step - loss: 0.1334 - val_loss: 0.1130
Epoch 5/40
61/61 [==============================] - 0s 5ms/step - loss: 0.1123 - val_loss: 0.1119
Epoch 6/40
61/61 [==============================] - 0s 5ms/step - loss: 0.1151 - val_loss: 0.1094
Epoch 7/40
61/61 [==============================] - 0s 7ms/step - loss: 0.1127 - val_loss: 0.1069
Epoch 8/40
61/61 [==============================] - 0s 5ms/step - loss: 0.1125 - val_loss: 0.1085
Epoch 9/40
61/61 [==============================] - 0s 5ms/step - loss: 0.1062 - val_loss: 0.1049
Epoch 10/40
61/61 [==============================] - 0s 5ms/step - loss: 0.1072 - val_

62/62 [==============================] - 0s 5ms/step - loss: 0.1495 - val_loss: 0.0817
Epoch 6/40
62/62 [==============================] - 0s 5ms/step - loss: 0.1418 - val_loss: 0.0792
Epoch 7/40
62/62 [==============================] - 0s 5ms/step - loss: 0.1373 - val_loss: 0.0778
Epoch 8/40
62/62 [==============================] - 0s 5ms/step - loss: 0.1339 - val_loss: 0.0798
Epoch 9/40
62/62 [==============================] - 0s 5ms/step - loss: 0.1335 - val_loss: 0.0799
Epoch 10/40
62/62 [==============================] - 0s 5ms/step - loss: 0.1390 - val_loss: 0.0822
Epoch 11/40
62/62 [==============================] - 0s 5ms/step - loss: 0.1264 - val_loss: 0.0847
Epoch 12/40
62/62 [==============================] - 0s 5ms/step - loss: 0.1343 - val_loss: 0.0799
Epoch 13/40
62/62 [==============================] - 0s 5ms/step - loss: 0.1352 - val_loss: 0.0830
Epoch 14/40
62/62 [==============================] - 0s 4ms/step - loss: 0.1337 - val_loss: 0.0836
Epoch 15/40
62/62 [=======

61/61 [==============================] - 0s 5ms/step - loss: 0.1185 - val_loss: 0.0849
Epoch 10/40
61/61 [==============================] - 0s 5ms/step - loss: 0.1158 - val_loss: 0.0881
Epoch 11/40
61/61 [==============================] - 0s 5ms/step - loss: 0.1126 - val_loss: 0.0805
Epoch 12/40
61/61 [==============================] - 0s 5ms/step - loss: 0.1124 - val_loss: 0.0755
Epoch 13/40
61/61 [==============================] - 0s 5ms/step - loss: 0.1070 - val_loss: 0.0767
Epoch 14/40
61/61 [==============================] - 0s 5ms/step - loss: 0.1070 - val_loss: 0.0817
Epoch 15/40
61/61 [==============================] - 0s 5ms/step - loss: 0.1034 - val_loss: 0.0764
Epoch 16/40
61/61 [==============================] - 0s 5ms/step - loss: 0.1018 - val_loss: 0.0766
Epoch 17/40
61/61 [==============================] - 0s 5ms/step - loss: 0.1034 - val_loss: 0.0784
Epoch 18/40
61/61 [==============================] - 0s 5ms/step - loss: 0.0992 - val_loss: 0.0771
Epoch 19/40
61/61 [===

62/62 [==============================] - 0s 5ms/step - loss: 0.0757 - val_loss: 0.0667
Epoch 14/40
62/62 [==============================] - 0s 5ms/step - loss: 0.0762 - val_loss: 0.0658
Epoch 15/40
62/62 [==============================] - 0s 5ms/step - loss: 0.0741 - val_loss: 0.0639
Epoch 16/40
62/62 [==============================] - 0s 5ms/step - loss: 0.0728 - val_loss: 0.0628
Epoch 17/40
62/62 [==============================] - 0s 5ms/step - loss: 0.0755 - val_loss: 0.0643
Epoch 18/40
62/62 [==============================] - 0s 5ms/step - loss: 0.0694 - val_loss: 0.0629
Epoch 19/40
62/62 [==============================] - 0s 5ms/step - loss: 0.0699 - val_loss: 0.0633
Epoch 20/40
62/62 [==============================] - 0s 5ms/step - loss: 0.0720 - val_loss: 0.0613
Epoch 21/40
62/62 [==============================] - 0s 5ms/step - loss: 0.0670 - val_loss: 0.0604
Epoch 22/40
62/62 [==============================] - 0s 5ms/step - loss: 0.0712 - val_loss: 0.0599
Epoch 23/40
62/62 [===

44/44 [==============================] - 0s 5ms/step - loss: 0.1035 - val_loss: 0.0562
Epoch 18/40
44/44 [==============================] - 0s 5ms/step - loss: 0.0988 - val_loss: 0.0557
Epoch 19/40
44/44 [==============================] - 0s 5ms/step - loss: 0.1017 - val_loss: 0.0540
Epoch 20/40
44/44 [==============================] - 0s 5ms/step - loss: 0.1019 - val_loss: 0.0565
Epoch 21/40
44/44 [==============================] - 0s 5ms/step - loss: 0.0983 - val_loss: 0.0523
Epoch 22/40
44/44 [==============================] - 0s 5ms/step - loss: 0.0963 - val_loss: 0.0528
Epoch 23/40
44/44 [==============================] - 0s 5ms/step - loss: 0.1004 - val_loss: 0.0518
Epoch 24/40
44/44 [==============================] - 0s 5ms/step - loss: 0.0955 - val_loss: 0.0506
Epoch 25/40
44/44 [==============================] - 0s 5ms/step - loss: 0.0969 - val_loss: 0.0498
Epoch 26/40
44/44 [==============================] - 0s 5ms/step - loss: 0.0973 - val_loss: 0.0515
Epoch 27/40
44/44 [===

...

47/47 [==============================] - 0s 4ms/step - loss: 0.0776 - val_loss: 0.0635
Epoch 18/40
47/47 [==============================] - 0s 4ms/step - loss: 0.0806 - val_loss: 0.0623
Epoch 19/40
47/47 [==============================] - 0s 4ms/step - loss: 0.0777 - val_loss: 0.0639
Epoch 20/40
47/47 [==============================] - 0s 4ms/step - loss: 0.0759 - val_loss: 0.0629
Epoch 21/40
47/47 [==============================] - 0s 4ms/step - loss: 0.0788 - val_loss: 0.0645
Epoch 22/40
47/47 [==============================] - 0s 4ms/step - loss: 0.0758 - val_loss: 0.0664
Epoch 23/40
47/47 [==============================] - 0s 4ms/step - loss: 0.0788 - val_loss: 0.0654
Epoch 24/40
47/47 [==============================] - 0s 4ms/step - loss: 0.0742 - val_loss: 0.0659
Epoch 25/40
47/47 [==============================] - 0s 4ms/step - loss: 0.0803 - val_loss: 0.0631
Epoch 26/40
47/47 [==============================] - 0s 4ms/step - loss: 0.0726 - val_loss: 0.0636
Epoch 27/40
47/47 [===

56/56 [==============================] - 0s 4ms/step - loss: 0.1168 - val_loss: 0.1011
Epoch 22/40
56/56 [==============================] - 0s 4ms/step - loss: 0.1149 - val_loss: 0.0973
Epoch 23/40
56/56 [==============================] - 0s 4ms/step - loss: 0.1132 - val_loss: 0.1036
Epoch 24/40
56/56 [==============================] - 0s 4ms/step - loss: 0.1156 - val_loss: 0.0966
Epoch 25/40
56/56 [==============================] - 0s 4ms/step - loss: 0.1102 - val_loss: 0.0977
Epoch 26/40
56/56 [==============================] - 0s 4ms/step - loss: 0.1144 - val_loss: 0.0961
Epoch 27/40
56/56 [==============================] - 0s 4ms/step - loss: 0.1131 - val_loss: 0.0943
Epoch 28/40
56/56 [==============================] - 0s 4ms/step - loss: 0.1185 - val_loss: 0.0976
Epoch 29/40
56/56 [==============================] - 0s 4ms/step - loss: 0.1123 - val_loss: 0.0960
Epoch 30/40
56/56 [==============================] - 0s 4ms/step - loss: 0.1114 - val_loss: 0.0966
Epoch 31/40
56/56 [===

62/62 [==============================] - 0s 4ms/step - loss: 0.0959 - val_loss: 0.0739
Epoch 26/40
62/62 [==============================] - 0s 4ms/step - loss: 0.0963 - val_loss: 0.0739
Epoch 27/40
62/62 [==============================] - 0s 4ms/step - loss: 0.0923 - val_loss: 0.0722
Epoch 28/40
62/62 [==============================] - 0s 5ms/step - loss: 0.0944 - val_loss: 0.0722
Epoch 29/40
62/62 [==============================] - 0s 4ms/step - loss: 0.0920 - val_loss: 0.0716
Epoch 30/40
62/62 [==============================] - 0s 5ms/step - loss: 0.0940 - val_loss: 0.0726
Epoch 31/40
62/62 [==============================] - 0s 4ms/step - loss: 0.0931 - val_loss: 0.0721
Epoch 32/40
62/62 [==============================] - 0s 4ms/step - loss: 0.0930 - val_loss: 0.0726
Epoch 33/40
62/62 [==============================] - 0s 4ms/step - loss: 0.0899 - val_loss: 0.0723
Epoch 34/40
62/62 [==============================] - 0s 4ms/step - loss: 0.0901 - val_loss: 0.0723
Epoch 35/40
62/62 [===

63/63 [==============================] - 0s 4ms/step - loss: 0.0878 - val_loss: 0.0881
Epoch 30/40
63/63 [==============================] - 0s 4ms/step - loss: 0.0889 - val_loss: 0.0858
Epoch 31/40
63/63 [==============================] - 0s 4ms/step - loss: 0.0899 - val_loss: 0.0828
Epoch 32/40
63/63 [==============================] - 0s 4ms/step - loss: 0.0900 - val_loss: 0.0874
Epoch 33/40
63/63 [==============================] - 0s 4ms/step - loss: 0.0880 - val_loss: 0.0886
Epoch 34/40
63/63 [==============================] - 0s 4ms/step - loss: 0.0918 - val_loss: 0.0875
Epoch 35/40
63/63 [==============================] - 0s 4ms/step - loss: 0.0881 - val_loss: 0.0836
Epoch 36/40
63/63 [==============================] - 0s 4ms/step - loss: 0.0898 - val_loss: 0.0829
Epoch 37/40
63/63 [==============================] - 0s 4ms/step - loss: 0.0898 - val_loss: 0.0876
Epoch 38/40
63/63 [==============================] - 0s 4ms/step - loss: 0.0858 - val_loss: 0.0889
Epoch 39/40
63/63 [===

62/62 [==============================] - 0s 4ms/step - loss: 0.1399 - val_loss: 0.1179
Epoch 34/40
62/62 [==============================] - 0s 4ms/step - loss: 0.1366 - val_loss: 0.1179
Epoch 35/40
62/62 [==============================] - 0s 4ms/step - loss: 0.1386 - val_loss: 0.1174
Epoch 36/40
62/62 [==============================] - 0s 4ms/step - loss: 0.1435 - val_loss: 0.1181
Epoch 37/40
62/62 [==============================] - 0s 4ms/step - loss: 0.1382 - val_loss: 0.1174
Epoch 38/40
62/62 [==============================] - 0s 4ms/step - loss: 0.1416 - val_loss: 0.1177
Epoch 39/40
62/62 [==============================] - 0s 4ms/step - loss: 0.1406 - val_loss: 0.1186
Epoch 40/40
62/62 [==============================] - 0s 4ms/step - loss: 0.1385 - val_loss: 0.1173
0.3876188479670073


In [17]:
print(outliers2.shape)
outliers2.head()

,1995-01-01,1995-01-02,1995-01-03,1995-01-04,1995-01-05,1995-01-06,1995-01-07,1995-01-08,1995-01-09,1995-01-10,...,2012-11-21,2012-11-22,2012-11-23,2012-11-24,2012-11-25,2012-11-26,2012-11-27,2012-11-28,2012-11-29,2012-11-30
proj_id,,,,,,,,,,,,,,,,,,,,,
BB_36485178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
BB_36490283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BB_36491640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BB_36500580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BB_36501649,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# check
outliers = pd.read_csv("dataset/outliers/outliers2.csv").set_index("proj_id")
outliers.head()

,1995-01-01,1995-01-02,1995-01-03,1995-01-04,1995-01-05,1995-01-06,1995-01-07,1995-01-08,1995-01-09,1995-01-10,...,2012-11-21,2012-11-22,2012-11-23,2012-11-24,2012-11-25,2012-11-26,2012-11-27,2012-11-28,2012-11-29,2012-11-30
proj_id,,,,,,,,,,,,,,,,,,,,,
BB_36485178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
BB_36490283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BB_36491640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BB_36500580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BB_36501649,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# check for one ts
one_ts = pd.DataFrame((grouped_t_s.iloc[0].dropna(),outliers.iloc[0].dropna())).T
one_ts

,BB_25470023,BB_36485178
length,567,NaN
freq,7,NaN
2000-10-17,79.01,NaN
2000-11-22,78.96,NaN
2000-12-18,78.94,NaN
...,...,...
2012-11-26,NaN,False
2012-11-27,NaN,False
2012-11-28,NaN,False
2012-11-29,NaN,False


In [41]:
one_ts[one_ts["anomaly"].notnull()]

,BB_25470023,anomaly
2000-10-17,79.01,False
2000-11-22,78.96,False
2000-12-18,78.94,False
2001-01-11,79.02,False
2001-02-06,78.99,False
...,...,...
2012-10-22,79.64,False
2012-11-01,79.64,False
2012-11-08,79.64,False
2012-11-15,79.62,False


In [40]:
one_ts.columns= [one_ts.columns[0],"anomaly"]
one_ts[one_ts["anomaly"]==True]
#one_ts

,BB_25470023,anomaly
2007-06-22,78.42,True
2011-08-01,79.51,True


,BB_25470023,anomaly
length,567,NaN
freq,7,NaN
2000-10-17,79.01,False
2000-11-22,78.96,False
2000-12-18,78.94,False
...,...,...
2012-10-22,79.64,False
2012-11-01,79.64,False
2012-11-08,79.64,False
2012-11-15,79.62,False
